# Project description
You work for the online store Ice, which sells video games all over the world. User and expert reviews, genres, platforms (e.g. Xbox or PlayStation), and historical data on game sales are available from open sources. You need to identify patterns that determine whether a game succeeds or not. This will allow you to spot potential big winners and plan advertising campaigns.

In front of you is data going back to 2016. Let’s imagine that it’s December 2016 and you’re planning a campaign for 2017. 

(The important thing is to get experience working with data. It doesn't really matter whether you're forecasting 2017 sales based on data from 2016 or 2027 sales based on data from 2026.)

The dataset contains the abbreviation ESRB. The Entertainment Software Rating Board evaluates a game's content and assigns an age rating such as Teen or Mature.


Data description
—Name 

—Platform 

—Year_of_Release 

—Genre 

—NA_sales (North American sales in USD million) 

—EU_sales (sales in Europe in USD million) 

—JP_sales (sales in Japan in USD million) 

—Other_sales (sales in other countries in USD million) 

—Critic_Score (maximum of 100) 

—User_Score (maximum of 10) 

—Rating (ESRB)

Data for 2016 may be incomplete.

In [126]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
from matplotlib import pyplot as plt
import altair


In [127]:
df = pd.read_csv('games.csv')



In [128]:
df.columns = df.columns.str.lower()
# Making all column headers lower case for easier analsysis
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [129]:
df.columns = df.columns.str.lower().str.replace(' ','')
# making certain theres no space between commas in the column headers

In [130]:
#df.info()
value_counts = df['user_score'].value_counts()

value_counts

user_score
tbd    2424
7.8     324
8       290
8.2     282
8.3     254
       ... 
1.1       2
1.9       2
9.6       2
0         1
9.7       1
Name: count, Length: 96, dtype: int64

In [131]:
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')
# Converting the user_score column to floats, I'm using the errors= 'coerce' parameter to convert all of the 'tbd' values and whatever values that couldn't be converted to floats to NaN values.
df['user_score'].isna().sum()

9125

Converting the user_score to float type because in the future I will most likely have to analyze the ratings scores and I can't do any sort of numerical analsysis unless the datatype is of float or int type.

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


In [133]:
df['user_score'].duplicated().sum()
# No duplicated rows within the entire dataset

16619

In [134]:
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [135]:
def count_mismatched_rows(df):
    
    col1, col2 = 'critic_score', 'user_score'
    
    # Conditions where one column is NaN and the other is not
   
     # Created a boolean where each element is true if the value in col1 is NaN and the value in col2 is not missing.
    condition1 = df[col1].isna() & df[col2].notna()
    
    # Created a boolean where each element is true if the value in col1 is not missing and the value in col2 is missing.
    condition2 = df[col1].notna() & df[col2].isna()
    
    
    # Count rows that match the conditions
    # The shape attribute returns a tuple representing the number of rows and the second element is the number of columns and I used 0 to indicate that were only looking at rows
    
    # This line will count the number of times condition1 and condition2 happen for rows only and the pipeline is used to combine the two conditions meaning a row will be counted if either condition1 is true or condition2 is true
    
    mismatch_count = df[condition1 | condition2].shape[0]
  
    return mismatch_count


mismatched_rows_count = count_mismatched_rows(df)

print(f"Number of rows with a value in one column but not the other: {mismatched_rows_count}")

Number of rows with a value in one column but not the other: 1693


In [136]:
df['name'].nunique()

11559

## Missing Value Analysis
There are many missing values from this dataset. The name column has two missing values which could be because the games ina different language and the inputters of the data just couldn't figure out the name or of course they could have just forgtten to put their names in. For our numerical data none of the sales columns having missing values however many have values of 0 indicating there were no sales on those games. For the score columns most likely many of the games were simply not given scores and for the rating columns many of which were not given a rating. 

I also decided to keep the NaN values and fill in using the median, there's too many missing values and I feel adding in median values for each one would skew the data in the wrong direction. Particucular regions may have very few values and some may have way more and if we were to fill in those missing values with the median it may drastically bump up or bring down those regions. I may in the future add median values for each particular region in seperate DataFrames in the future. It will also screw up my standard deviation, my distribution would be completely skewed towards the center with how many missing values there are to be replaced which is around 8,000. 




In [137]:
# This code adds up all the columns we indicated it to and used sum with the axis parameters to indicate we wanted to add each column we told it to together. 

# I need to remember to plug two sets of brackets around a list of column names, if it was one column I'd only use one set of brackets.
df['total_sales'] = df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38


In [138]:
df = df.sort_values(by= 'year_of_release', ascending= True)

df

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
1764,Kaboom!,2600,1980.0,Misc,1.07,0.07,0.0,0.01,NaN,NaN,NaN,1.15
546,Missile Command,2600,1980.0,Shooter,2.56,0.17,0.0,0.03,NaN,NaN,NaN,2.76
1968,Defender,2600,1980.0,Misc,0.99,0.05,0.0,0.01,NaN,NaN,NaN,1.05
6300,Bridge,2600,1980.0,Misc,0.25,0.02,0.0,0.00,NaN,NaN,NaN,0.27
6875,Checkers,2600,1980.0,Misc,0.22,0.01,0.0,0.00,NaN,NaN,NaN,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
16373,PDC World Championship Darts 2008,PSP,NaN,Sports,0.01,0.00,0.0,0.00,43.0,NaN,E10+,0.01
16405,Freaky Flyers,GC,NaN,Racing,0.01,0.00,0.0,0.00,69.0,6.5,T,0.01
16448,Inversion,PC,NaN,Shooter,0.01,0.00,0.0,0.00,59.0,6.7,M,0.01
16458,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,0.01,0.00,0.0,0.00,NaN,NaN,NaN,0.01


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16715 entries, 1764 to 16522
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
 11  total_sales      16715 non-null  float64
dtypes: float64(8), object(4)
memory usage: 1.7+ MB


In [140]:
df['year_of_release'].unique()

# Creating a new DataFrame of year_of_release and the count of the rows (games) in each group (year)
games_per_year = df.groupby('year_of_release').size().reset_index(name='Number of Games per Year')

games_per_year = games_per_year.sort_values(by= 'Number of Games per Year', ascending= True )

games_per_year.head()



,year_of_release,Number of Games per Year
0,1980.0,9
4,1984.0,14
5,1985.0,14
8,1988.0,15
10,1990.0,16


In [141]:
fig = px.bar(games_per_year, x='year_of_release', y='Number of Games per Year',
             title='Number of Games Released per Year',
             labels={'year_of_release': 'Year of Release', 'Number of Games per year': 'Number of Games'},
             color='Number of Games per Year')
fig.show()

# Number of Games Produced Each Year (bar chart)


The most games produced in a year is in 2008 with 1,427 games produced

The least is 1980 with only 9 games produced

In [142]:
df['platform'].unique()

array(['2600', 'NES', 'DS', 'PC', 'GB', 'SNES', 'GEN', 'GG', 'NG', 'SCD',
       'SAT', 'PS', '3DO', 'TG16', 'N64', 'PCFX', 'DC', 'WS', 'PS2',
       'GBA', 'XB', 'GC', 'PSP', 'X360', 'PS3', 'Wii', '3DS', 'PSV',
       'WiiU', 'XOne', 'PS4'], dtype=object)

In [144]:
sales_df = df[['name', 'year_of_release', 'platform', 'total_sales']]

sales_df

,name,year_of_release,platform,total_sales
1764,Kaboom!,1980.0,2600,1.15
546,Missile Command,1980.0,2600,2.76
1968,Defender,1980.0,2600,1.05
6300,Bridge,1980.0,2600,0.27
6875,Checkers,1980.0,2600,0.23
...,...,...,...,...
16373,PDC World Championship Darts 2008,NaN,PSP,0.01
16405,Freaky Flyers,NaN,GC,0.01
16448,Inversion,NaN,PC,0.01
16458,Hakuouki: Shinsengumi Kitan,NaN,PS3,0.01


In [148]:
total_sales_by_platform = sales_df.groupby('platform')['total_sales'].sum().reset_index(name='total_platform_sales')

total_sales_by_platform = total_sales_by_platform.sort_values(by='total_platform_sales', ascending= False)

total_sales_by_platform.head()

,platform,total_platform_sales
16,PS2,1255.77
28,X360,971.42
17,PS3,939.65
26,Wii,907.51
4,DS,806.12


In [170]:
top_platforms = total_sales_by_platform['platform'].head(5)  


yearly_sales = sales_df[sales_df['platform'].isin(top_platforms)].groupby(['year_of_release', 'platform'])['total_sales'].sum().reset_index()

DS = yearly_sales[yearly_sales['platform'] == 'DS']
DS


,year_of_release,platform,total_sales
0,1985.0,DS,0.02
5,2004.0,DS,17.27
7,2005.0,DS,130.14
10,2006.0,DS,119.81
15,2007.0,DS,146.94
20,2008.0,DS,145.31
25,2009.0,DS,119.54
30,2010.0,DS,85.02
35,2011.0,DS,26.18
40,2012.0,DS,11.01


In [171]:
fig = px.line(yearly_sales, x='year_of_release', y='total_sales', color='platform',
              title='Yearly Total Sales for Top Platforms',
              labels={'year_of_release': 'Year of Release', 'total_sales': 'Total Sales'},
              markers=True)

# Show the plot
fig.show()

# Sales for each Platform Overtime (line graph)

You can see the trend for each indvidual platform regarding sales on this line graph.

DS for example goes from sales of 17.27 million in 2004 to 130.14 million in 2005 however DS's sales have been decreasing from 2007 on. 


In [174]:
platform_lifecycle = sales_df.groupby('platform')['year_of_release'].agg(first_year='min', last_year='max').reset_index()
platform_lifecycle.head()

,platform,first_year,last_year
0,2600,1980.0,1989.0
1,3DO,1994.0,1995.0
2,3DS,2011.0,2016.0
3,DC,1998.0,2008.0
4,DS,1985.0,2013.0


In [177]:
platform_lifecycle['lifecycle_duration'] = platform_lifecycle['last_year'] - platform_lifecycle['first_year']
platform_lifecycle.head()

,platform,first_year,last_year,lifecycle_duration
0,2600,1980.0,1989.0,9.0
1,3DO,1994.0,1995.0,1.0
2,3DS,2011.0,2016.0,5.0
3,DC,1998.0,2008.0,10.0
4,DS,1985.0,2013.0,28.0


In [184]:
platform_lifecycle['lifecycle_duration'].describe()

count    31.000000
mean      7.612903
std       6.998464
min       0.000000
25%       3.000000
50%       6.000000
75%      10.000000
max      31.000000
Name: lifecycle_duration, dtype: float64

In [185]:
import plotly.express as px

fig = px.histogram(platform_lifecycle, x='lifecycle_duration',
                   title='Distribution of Platform Lifecycles',
                   labels={'lifecycle_duration': 'Lifecycle Duration (Years)'},
                   nbins=30)  
fig.show()

# Distribution of Platform Lifecycles 

The average lifecycle for platforms are around 7 years on average.

The most common lifespan cycle for platforms is between 10-11 years.

 
